<a href="https://colab.research.google.com/github/aasimsani/model-quick-deploy/blob/main/Model_Quick_Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install requirements
!pip install fastapi==0.68.1
!pip install opencv-python==4.5.3.56
!pip insall Pillow==8.3.2
!pip install timm==0.4.12
!pip install python-multipart==0.0.5
!pip install uvicorn==0.15.0

     |████████████████████████████████| 52 kB 716 kB/s 
     |████████████████████████████████| 10.1 MB 9.6 MB/s 
     |████████████████████████████████| 60 kB 6.6 MB/s 
     |████████████████████████████████| 49.9 MB 16 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
ERROR: unknown command "insall" - maybe you meant "install"
     |████████████████████████████████| 376 kB 5.2 MB/s 
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=3df66d19143f842e2f83bbd7e95e8845128e057802ab84b1bc81b83977e66c4b
  Stored in director

In [2]:
!pip install nest-asyncio

In [3]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=1f7dcdaba08963cecf5b38a467f02d5d651c9130974bbd2dd3da1ea6fe7795a9
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [26]:
from typing import List

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse, StreamingResponse

import cv2
import io
import numpy as np

import torch
import cv2
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

## Load model

# MiDaS v3 - Large
# (highest accuracy, slowest inference speed)

# model_type = "DPT_Large"

# MiDaS v3 - Hybrid
# (medium accuracy, medium inference speed)
# model_type = "DPT_Hybrid"

# (lowest accuracy, highest inference speed)
model_type = "MiDaS_small"  # MiDaS v2.1 - Small

midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# Code from: https://fastapi.tiangolo.com/tutorial/request-files/
app = FastAPI()


@app.post("/uploadfiles/")
async def create_upload_files(files: List[UploadFile] = File(...)):
    """ Create API endpoint to send image to and specify
     what type of file it'll take

    :param files: Get image files, defaults to File(...)
    :type files: List[UploadFile], optional
    :return: A list of png images
    :rtype: list(bytes)
    """

    for image in files:

        # Load image
        img = cv2.imdecode(np.frombuffer(image.file.read(),
                                         np.uint8),
                           cv2.IMREAD_COLOR)

        # convert it to the correct format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Transform it so that it can be used by the model
        input_batch = transform(img).to(device)

        # Run the model and postpocess the output
        with torch.no_grad():
            prediction = midas(input_batch)

            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

        output = prediction.cpu().numpy()

        # Create a figure using matplotlib which super-imposes the original
        # image and the prediction

        fig = Figure()
        canvas = FigureCanvas(fig)
        ax = fig.gca()

        # Render both images original as foreground
        ax.imshow(img)
        ax.imshow(output, cmap="jet", alpha=0.8)

        ax.axis("off")
        canvas.draw()

        # Reshape output to be a numpy array
        width, height = fig.get_size_inches() * fig.get_dpi()
        width = int(width)
        height = int(height)
        output_image = np.frombuffer(canvas.tostring_rgb(),
                                     dtype='uint8').reshape(height, width, 3)

        # Encode to png
        res, im_png = cv2.imencode(".png", output_image)
        return StreamingResponse(io.BytesIO(im_png.tobytes()),
                                 media_type="image/png")


@app.get("/")
async def main():
     """Create a basic home page to upload a file

    :return: HTML for homepage
    :rtype: HTMLResponse
    """

	content = """
	<body>
		<h3>Upload an image to get it's depth map from the MiDaS model</h3>
		<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
			<input name="files" type="file" multiple>
			<input type="submit">
		</form>
	</body>
	"""
	return HTMLResponse(content=content)


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


Running on  cuda


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [5]:
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
!ngrok authtoken <put auth token for ngrok here>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [27]:
from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000, port='8000')

In [28]:
# Check if it exists
!ps aux | grep ngrok

root         363 28.0  0.1 726652 25716 ?        Sl   05:19   0:00 /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root         373  0.0  0.0  39196  6580 ?        S    05:19   0:00 /bin/bash -c ps aux | grep ngrok
root         375  0.0  0.0  38576  5580 ?        S    05:19   0:00 grep ngrok


In [ ]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

NgrokTunnel: "http://cf22-34-69-236-155.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [79]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     66.249.72.64:0 - "GET /robots.txt HTTP/1.1" 404 Not Found
INFO:     66.249.72.64:0 - "GET / HTTP/1.1" 200 OK
INFO:     64.53.1.70:0 - "GET / HTTP/1.1" 200 OK
INFO:     69.249.206.223:0 - "GET / HTTP/1.1" 200 OK
INFO:     69.249.206.223:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     198.7.237.196:0 - "GET / HTTP/1.1" 200 OK
INFO:     67.219.192.37:0 - "GET /uploadfiles/ HTTP/1.1" 405 Method Not Allowed


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


INFO:     69.249.206.223:0 - "POST /uploadfiles/ HTTP/1.1" 200 OK


Streaming output truncated to the last 5000 lines.
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7effa1c17f50> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(17, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(17, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7effa1c17f50> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(17, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(17, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7effa1c17f50> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(17, 1)
handle: <Handle BaseAsyncIOLoop._handl

In [25]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)